In [1]:
import findspark
findspark.init('/home/osboxes/spark-2.4.3-bin-hadoop2.7')
import pyspark
import os

In [2]:
myPath = os.path.join('/home', 'osboxes', 'CourseMaterial', 'Spark_for_Machine_Learning', 'Linear_Regression')

print(myPath)

os.chdir(myPath)

/home/osboxes/CourseMaterial/Spark_for_Machine_Learning/Linear_Regression


In [3]:
os.listdir()

['cruise_ship_info.csv',
 '.ipynb_checkpoints',
 'Linear_Regression_Code_Along.ipynb',
 'Linear_Regression_Consulting_Project_SOLUTIONS.ipynb',
 'Ecommerce_Customers.csv',
 'Data_Transformations.ipynb',
 'fake_customers.csv',
 'sample_linear_regression_data.txt',
 'Linear_Regression_Example.ipynb',
 'Linear_Regression_Consulting_Project.ipynb']

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression

In [7]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [10]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [11]:
# print it
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [12]:
# set up the dataframe for machine learning

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler




In [14]:
# work with the numerical data only

data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [15]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
                                       'Time on App',
                                       'Time on Website',
                                       'Length of Membership'],outputCol='features')

In [16]:
output = assembler.transform(data)

In [19]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [20]:
# note that the key variables are in an array

output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [21]:
# select the data so that you get two columns
final_data = output.select('features', 'Yearly Amount Spent')

In [22]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [23]:
# split to train test

train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [24]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                356|
|   mean| 498.19000751919265|
| stddev|  79.81359338611833|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [25]:
test_data.describe().show()


+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                144|
|   mean| 502.09289203099183|
| stddev|  78.27422895603772|
|    min|  275.9184206503857|
|    max|  744.2218671047146|
+-------+-------------------+



In [26]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [30]:
lr_model = lr.fit(train_data)

In [31]:
test_results = lr_model.evaluate(test_data)

In [32]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 10.878792355831138|
| 10.277259115762376|
| -4.779659812319721|
| -4.308390910167361|
|  4.895468215297456|
| -21.92978524286525|
| 21.600745097017352|
| -5.467706217176101|
| -3.867611023262782|
|-18.355032913294053|
| -2.326026079384633|
|-1.8701514515932445|
|  -6.82722213562505|
| -4.859966620740863|
|-11.452358101945265|
|  7.727132673588983|
|-2.6097476612524133|
|-2.7183654596852875|
| 11.418220736869557|
|-13.919594891933684|
+-------------------+
only showing top 20 rows



In [33]:
test_results.rootMeanSquaredError

10.364423209951063

In [35]:
# get the r-squared

test_results.r2

0.9823445360478301

In [36]:
# deploy on data with only the features

unlabeled_data = test_data.select('features')

In [38]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.7377203726281...|
|[30.8364326747734...|
|[30.8794843441274...|
|[31.0472221394875...|
|[31.1239743499119...|
|[31.2834474760581...|
|[31.5147378578019...|
|[31.5171218025062...|
|[31.5702008293202...|
|[31.5761319713222...|
|[31.7216523605090...|
|[31.7242025238451...|
|[31.7656188210424...|
|[31.8093003166791...|
|[31.8209982016720...|
|[31.8530748017465...|
|[31.8627411090001...|
|[31.9096268275227...|
|[31.9365486184489...|
+--------------------+
only showing top 20 rows



In [39]:
predictions = lr_model.transform(unlabeled_data)

In [41]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|397.76155871679634|
|[30.7377203726281...| 451.5034830804675|
|[30.8364326747734...|472.28156023930933|
|[30.8794843441274...|494.51499089502204|
|[31.0472221394875...|387.60193097372394|
|[31.1239743499119...|  508.876839082631|
|[31.2834474760581...| 570.1803443286501|
|[31.5147378578019...| 495.2801942136375|
|[31.5171218025062...| 279.7860316736485|
|[31.5702008293202...| 564.3005250546989|
|[31.5761319713222...|  543.552610068713|
|[31.7216523605090...| 349.6470780834659|
|[31.7242025238451...|510.21510942358555|
|[31.7656188210424...|  501.414048256348|
|[31.8093003166791...| 548.2242574647864|
|[31.8209982016720...|416.94814833962437|
|[31.8530748017465...| 461.8948711236044|
|[31.8627411090001...|  559.016506633732|
|[31.9096268275227...| 552.0278149363696|
|[31.9365486184489...| 441.1189797872619|
+--------------------+------------